In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "."]).decode("utf8"))

comment-classification
comment-classification.zip
preprocess.ipynb
processed_x.csc
processed_x.csv
sentiment_analysis_trainingset.csv
sentiment_analysis_validationset.csv
train.ipynb
train_x_processed
train_y_processed
val_x_processed
val_y_processed



In [23]:
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [11]:
train_x_processed = pd.read_csv('train_x_processed')
train_y_processed = pd.read_csv('train_y_processed')
val_x_processed = pd.read_csv('val_x_processed')
val_y_processed = pd.read_csv('val_y_processed')

In [32]:
max_features = 10000
maxlen = 200

In [33]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_x_processed.content))

In [34]:
train_x_token = tokenizer.texts_to_sequences(train_x_processed.content)
val_x_token = tokenizer.texts_to_sequences(val_x_processed.content)

In [35]:
train_X = sequence.pad_sequences(train_x_token, maxlen=maxlen)
val_X = sequence.pad_sequences(val_x_token, maxlen=maxlen)

In [39]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [43]:
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.1))
model.add(Dense(1, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(80, activation="sigmoid"))
model.compile('adam', 'binary_crossentropy', metrics=['accuracy', f1])

In [44]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 200, 128)          1280000   
_________________________________________________________________
bidirectional_5 (Bidirection (None, 200, 128)          98816     
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 128)               0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 129       
_________________________________________________________________
dropout_10 (Dropout)         (None, 1)                 0         
_________________________________________________________________
dense_10 (Dense)             (None, 80)                160       
Total para

In [48]:
batch_size = 32
epochs = 4

model.fit(train_X, train_y_processed,
          batch_size=batch_size,
          epochs=4,
          verbose=1,
          validation_data=[val_X, val_y_processed])

Train on 105000 samples, validate on 15000 samples
Epoch 1/4
105000/105000 [==============================] - 811s 8ms/step - loss: 0.3399 - acc: 0.8529 - f1: 0.6882 - val_loss: 0.3367 - val_acc: 0.8555 - val_f1: 0.6940
Epoch 2/4
105000/105000 [==============================] - 780s 7ms/step - loss: 0.3384 - acc: 0.8535 - f1: 0.6896 - val_loss: 0.3368 - val_acc: 0.8555 - val_f1: 0.6927
Epoch 3/4
105000/105000 [==============================] - 795s 8ms/step - loss: 0.3373 - acc: 0.8541 - f1: 0.6907 - val_loss: 0.3367 - val_acc: 0.8557 - val_f1: 0.6957
Epoch 4/4
105000/105000 [==============================] - 781s 7ms/step - loss: 0.3361 - acc: 0.8546 - f1: 0.6919 - val_loss: 0.3372 - val_acc: 0.8553 - val_f1: 0.6938


In [49]:
model.save("./models/base_8epoch.h5")